# Setup codes

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent
from aiida.backends.utils import get_automatic_user, get_backend_type


import ipywidgets as ipw
from IPython.display import display, clear_output

from subprocess import check_call, check_output, call
from os import path
import re
import pexpect

In [ ]:
layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

In [ ]:
q = QueryBuilder()
q.append(Computer)
computers = [(sublist[0].name, sublist[0].name) for sublist in q.all() ]
computers.insert(0,('select', None))

In [ ]:
drop_computer = ipw.Dropdown(description="Select Computer:", options=computers, style=style, layout=layout )
display(drop_computer)

In [ ]:
def setup_codes(computer_name, partition):
    computer = Computer.get(computer_name)
    
    qe_path = "/ssoft/spack/cornalin/v1/opt/spack/linux-rhel7-x86_E5v2_IntelIB/intel-17.0.2/espresso-6.1.0-qmy7wm3lqqyphqouvajxjzqbetoopakn/bin/"
    
    qe_version = "6.1"
    for codename in ['pw', 'ph', 'pp', 'projwfc', 'matdyn', 'q2r']:
        # QuantumESPRESSO code
        code = Code(remote_computer_exec=(computer, qe_path + "{}.x".format(codename)))
        code.label = "{}-{}".format(codename, qe_version)
        code.description = "Quantum ESPRESSO {}.x".format(codename)
        code.set_input_plugin_name("quantumespresso.{}".format(qe_version))
        prepend_text  = "### code prepend_text start ###\n"
        prepend_text += """module load intel
module load intel-mpi
module load intel-mkl
module load espresso/6.1.0-mpi
"""
        prepend_text += "### code prepend_text end ###\n"
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        code._reveal()
        full_string = "{}-{}@{}".format(codename, qe_version, computer_name)
        ! verdi code show "{full_string}"

In [ ]:
def on_setup_codes(b):
    with setup_code_out:
        clear_output()
        partition = drop_computer.value
        if drop_computer.value is not None:
            setup_codes(drop_computer.value, partition)
        else:
            print ("You did not specify the computer name")

setup_code_out = ipw.Output()
btn_setup_codes = ipw.Button(description="Setup Codes")
btn_setup_codes.on_click(on_setup_codes)
display(btn_setup_codes, setup_code_out)